Install dependencies

In [1]:
!sudo apt update && sudo apt install pciutils lshw -y
print("✅ Dependencies installed")

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,776 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,986 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,037 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubunt

Install Ollama

In [3]:
!curl -fsSL https://ollama.com/install.sh | sh
print("✅ Ollama installed")

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
✅ Ollama installed


Start Ollama server

In [ ]:
# ============================================================================
# ENHANCED COLAB SESSION MANAGEMENT & TIMEOUT PREVENTION
# ============================================================================

import threading
import time
import requests
import json
from datetime import datetime, timedelta
import os

class ColabSessionManager:
    """Manages Colab session to prevent timeouts and keep it running longer"""
    
    def __init__(self):
        self.keep_alive = True
        self.heartbeat_interval = 300  # 5 minutes
        self.activity_thread = None
        self.last_activity = datetime.now()
        self.session_start = datetime.now()
        self.max_session_duration = 20 * 60  # 20 minutes default
        
    def start_keep_alive(self):
        """Start background thread to keep session alive"""
        if self.activity_thread is None or not self.activity_thread.is_alive():
            self.activity_thread = threading.Thread(target=self._keep_alive_worker, daemon=True)
            self.activity_thread.start()
            print("🔄 Session keep-alive started (prevents timeout)")
    
    def _keep_alive_worker(self):
        """Background worker to maintain session activity"""
        while self.keep_alive:
            try:
                # Simulate activity to prevent timeout
                self._simulate_activity()
                self.last_activity = datetime.now()
                
                # Check session duration
                session_duration = (datetime.now() - self.session_start).total_seconds()
                remaining_time = self.max_session_duration - session_duration
                
                if remaining_time > 0:
                    print(f"⏰ Session active for {session_duration/60:.1f} min, {remaining_time/60:.1f} min remaining")
                else:
                    print("⚠️ Session approaching maximum duration - consider restarting")
                
                time.sleep(self.heartbeat_interval)
                
            except Exception as e:
                print(f"⚠️ Keep-alive error: {e}")
                time.sleep(60)  # Wait before retrying
    
    def _simulate_activity(self):
        """Simulate minimal activity to prevent session timeout"""
        try:
            # Simple computation to show activity
            _ = sum(range(100))
            
            # Check if Ollama is still running
            try:
                import subprocess
                result = subprocess.run(['pgrep', 'ollama'], capture_output=True, text=True)
                if not result.stdout.strip():
                    print("🔄 Ollama process not found, restarting...")
                    self._restart_ollama()
            except:
                pass
                
        except Exception as e:
            print(f"Activity simulation error: {e}")
    
    def _restart_ollama(self):
        """Restart Ollama if it stops"""
        try:
            import subprocess
            # Kill any existing ollama processes
            subprocess.run(['pkill', 'ollama'], capture_output=True)
            time.sleep(2)
            
            # Start ollama serve in background
            subprocess.Popen(['ollama', 'serve'], 
                           stdout=subprocess.PIPE, 
                           stderr=subprocess.PIPE)
            time.sleep(5)
            print("✅ Ollama restarted")
        except Exception as e:
            print(f"❌ Failed to restart Ollama: {e}")
    
    def extend_session(self, additional_minutes=20):
        """Extend the maximum session duration"""
        self.max_session_duration += additional_minutes * 60
        print(f"⏰ Session extended by {additional_minutes} minutes")
    
    def get_session_info(self):
        """Get current session information"""
        session_duration = (datetime.now() - self.session_start).total_seconds()
        remaining_time = max(0, self.max_session_duration - session_duration)
        
        return {
            "session_duration_minutes": session_duration / 60,
            "remaining_time_minutes": remaining_time / 60,
            "last_activity": self.last_activity.isoformat(),
            "keep_alive_active": self.keep_alive and (self.activity_thread and self.activity_thread.is_alive())
        }
    
    def stop_keep_alive(self):
        """Stop the keep-alive mechanism"""
        self.keep_alive = False
        if self.activity_thread:
            self.activity_thread.join(timeout=5)
        print("🛑 Session keep-alive stopped")

# Initialize session manager
session_manager = ColabSessionManager()

print("✅ Enhanced session management initialized")
print("🔧 Features:")
print("   • Automatic session keep-alive")
print("   • Ollama process monitoring")
print("   • Session duration tracking")
print("   • Configurable timeout prevention")


In [4]:
import subprocess
import time

Start Ollama server in background

In [5]:
process = subprocess.Popen(['ollama', 'serve'],
                          stdout=subprocess.PIPE,
                          stderr=subprocess.PIPE)
print("🚀 Starting Ollama server...")
time.sleep(10)  # Wait for server to initialize
print("✅ Ollama server running")

🚀 Starting Ollama server...
✅ Ollama server running


Pull phi3:mini model

In [6]:
!ollama pull phi3:mini
print("✅ phi3:mini model downloaded")


✅ phi3:mini model downloaded


Verify everything is working


In [7]:
!ollama list

NAME         ID              SIZE      MODIFIED      
phi3:mini    4f2222927938    2.2 GB    6 seconds ago    


Test phi3:mini

In [8]:
!ollama run phi3:mini "Bonjour, tu peux m'aider avec des questions médicales?"

Oui, bien sûr! Je suis là pour répondre à vos questions aussi clairement que possible. Cependant, je ne puis pas recueillir de données personnelles ou fournir une assistance clinique en tant qu'assistant IA basé sur des informations préexistantes et non expertes dans la médecine comme un professionnel humain le serait. Si vous avez besoin d'informations, je peux tenter de les aider, mais pour tout symptôme sérieux ou urgent, il est toujours préférable d'agir avec l'aide d'un professionnel médical. Pourriez-vous me dire comment puis-je vous aider aujourd'hui?



# Medical Chatbot Core Implementation

Install required packages

In [1]:
!pip install ollama fastapi uvicorn pyngrok gradio mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 12.2 MB/s eta 0:00:00


In [ ]:
# Fix the ngrok URL format and test the API
from pyngrok import ngrok
import requests

# Get the clean URL
tunnels = ngrok.get_tunnels()
if tunnels:
    clean_url = tunnels[0].public_url
    print(f"🌐 Clean API URL: {clean_url}")

    # Test the API properly
    try:
        response = requests.get(f"{clean_url}/", timeout=10)
        if response.status_code == 200:
            print("✅ API test successful!")
            print(f"   Response: {response.json()}")
        else:
            print(f"⚠️ API returned status: {response.status_code}")
    except Exception as e:
        print(f"❌ API test failed: {e}")

    # Save the clean URL
    with open('api_url_clean.txt', 'w') as f:
        f.write(f"COLAB_API_URL={clean_url}\n")
        f.write(f"DOCS_URL={clean_url}/docs\n")
        f.write(f"HEALTH_URL={clean_url}/\n")
        f.write(f"CHAT_URL={clean_url}/chat\n")

    print(f"""
    ✅ FIXED! Use this clean URL:

    🌐 API URL: {clean_url}
    📖 Documentation: {clean_url}/docs

    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL={clean_url}
    COLAB_ENABLED=true
    """)
else:
    print("❌ No active tunnels found")

🌐 Clean API URL: https://f56b-34-125-17-230.ngrok-free.app
INFO:     34.125.17.230:0 - "GET / HTTP/1.1" 200 OK
✅ API test successful!
   Response: {'status': 'healthy', 'model': 'phi3:mini', 'server': 'Google Colab', 'timestamp': '2025-06-13T22:03:29.431286'}

    ✅ FIXED! Use this clean URL:
    
    🌐 API URL: https://f56b-34-125-17-230.ngrok-free.app
    📖 Documentation: https://f56b-34-125-17-230.ngrok-free.app/docs
    
    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL=https://f56b-34-125-17-230.ngrok-free.app
    COLAB_ENABLED=true
    


In [ ]:
# Fix the ngrok URL format and test the API
from pyngrok import ngrok
import requests

# Get the clean URL
tunnels = ngrok.get_tunnels()
if tunnels:
    clean_url = tunnels[0].public_url
    print(f"🌐 Clean API URL: {clean_url}")

    # Test the API properly
    try:
        response = requests.get(f"{clean_url}/", timeout=10)
        if response.status_code == 200:
            print("✅ API test successful!")
            print(f"   Response: {response.json()}")
        else:
            print(f"⚠️ API returned status: {response.status_code}")
    except Exception as e:
        print(f"❌ API test failed: {e}")

    # Save the clean URL
    with open('api_url_clean.txt', 'w') as f:
        f.write(f"COLAB_API_URL={clean_url}\n")
        f.write(f"DOCS_URL={clean_url}/docs\n")
        f.write(f"HEALTH_URL={clean_url}/\n")
        f.write(f"CHAT_URL={clean_url}/chat\n")

    print(f"""
    ✅ FIXED! Use this clean URL:

    🌐 API URL: {clean_url}
    📖 Documentation: {clean_url}/docs

    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL={clean_url}
    COLAB_ENABLED=true
    """)
else:
    print("❌ No active tunnels found")

🌐 Clean API URL: https://f56b-34-125-17-230.ngrok-free.app
INFO:     34.125.17.230:0 - "GET / HTTP/1.1" 200 OK
✅ API test successful!
   Response: {'status': 'healthy', 'model': 'phi3:mini', 'server': 'Google Colab', 'timestamp': '2025-06-13T22:03:29.431286'}

    ✅ FIXED! Use this clean URL:
    
    🌐 API URL: https://f56b-34-125-17-230.ngrok-free.app
    📖 Documentation: https://f56b-34-125-17-230.ngrok-free.app/docs
    
    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL=https://f56b-34-125-17-230.ngrok-free.app
    COLAB_ENABLED=true
    


In [ ]:
# Fix the ngrok URL format and test the API
from pyngrok import ngrok
import requests

# Get the clean URL
tunnels = ngrok.get_tunnels()
if tunnels:
    clean_url = tunnels[0].public_url
    print(f"🌐 Clean API URL: {clean_url}")

    # Test the API properly
    try:
        response = requests.get(f"{clean_url}/", timeout=10)
        if response.status_code == 200:
            print("✅ API test successful!")
            print(f"   Response: {response.json()}")
        else:
            print(f"⚠️ API returned status: {response.status_code}")
    except Exception as e:
        print(f"❌ API test failed: {e}")

    # Save the clean URL
    with open('api_url_clean.txt', 'w') as f:
        f.write(f"COLAB_API_URL={clean_url}\n")
        f.write(f"DOCS_URL={clean_url}/docs\n")
        f.write(f"HEALTH_URL={clean_url}/\n")
        f.write(f"CHAT_URL={clean_url}/chat\n")

    print(f"""
    ✅ FIXED! Use this clean URL:

    🌐 API URL: {clean_url}
    📖 Documentation: {clean_url}/docs

    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL={clean_url}
    COLAB_ENABLED=true
    """)
else:
    print("❌ No active tunnels found")

🌐 Clean API URL: https://f56b-34-125-17-230.ngrok-free.app
INFO:     34.125.17.230:0 - "GET / HTTP/1.1" 200 OK
✅ API test successful!
   Response: {'status': 'healthy', 'model': 'phi3:mini', 'server': 'Google Colab', 'timestamp': '2025-06-13T22:03:29.431286'}

    ✅ FIXED! Use this clean URL:
    
    🌐 API URL: https://f56b-34-125-17-230.ngrok-free.app
    📖 Documentation: https://f56b-34-125-17-230.ngrok-free.app/docs
    
    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL=https://f56b-34-125-17-230.ngrok-free.app
    COLAB_ENABLED=true
    


In [ ]:
# Fix the ngrok URL format and test the API
from pyngrok import ngrok
import requests

# Get the clean URL
tunnels = ngrok.get_tunnels()
if tunnels:
    clean_url = tunnels[0].public_url
    print(f"🌐 Clean API URL: {clean_url}")

    # Test the API properly
    try:
        response = requests.get(f"{clean_url}/", timeout=10)
        if response.status_code == 200:
            print("✅ API test successful!")
            print(f"   Response: {response.json()}")
        else:
            print(f"⚠️ API returned status: {response.status_code}")
    except Exception as e:
        print(f"❌ API test failed: {e}")

    # Save the clean URL
    with open('api_url_clean.txt', 'w') as f:
        f.write(f"COLAB_API_URL={clean_url}\n")
        f.write(f"DOCS_URL={clean_url}/docs\n")
        f.write(f"HEALTH_URL={clean_url}/\n")
        f.write(f"CHAT_URL={clean_url}/chat\n")

    print(f"""
    ✅ FIXED! Use this clean URL:

    🌐 API URL: {clean_url}
    📖 Documentation: {clean_url}/docs

    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL={clean_url}
    COLAB_ENABLED=true
    """)
else:
    print("❌ No active tunnels found")

🌐 Clean API URL: https://f56b-34-125-17-230.ngrok-free.app
INFO:     34.125.17.230:0 - "GET / HTTP/1.1" 200 OK
✅ API test successful!
   Response: {'status': 'healthy', 'model': 'phi3:mini', 'server': 'Google Colab', 'timestamp': '2025-06-13T22:03:29.431286'}

    ✅ FIXED! Use this clean URL:
    
    🌐 API URL: https://f56b-34-125-17-230.ngrok-free.app
    📖 Documentation: https://f56b-34-125-17-230.ngrok-free.app/docs
    
    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL=https://f56b-34-125-17-230.ngrok-free.app
    COLAB_ENABLED=true
    


In [ ]:
# Fix the ngrok URL format and test the API
from pyngrok import ngrok
import requests

# Get the clean URL
tunnels = ngrok.get_tunnels()
if tunnels:
    clean_url = tunnels[0].public_url
    print(f"🌐 Clean API URL: {clean_url}")

    # Test the API properly
    try:
        response = requests.get(f"{clean_url}/", timeout=10)
        if response.status_code == 200:
            print("✅ API test successful!")
            print(f"   Response: {response.json()}")
        else:
            print(f"⚠️ API returned status: {response.status_code}")
    except Exception as e:
        print(f"❌ API test failed: {e}")

    # Save the clean URL
    with open('api_url_clean.txt', 'w') as f:
        f.write(f"COLAB_API_URL={clean_url}\n")
        f.write(f"DOCS_URL={clean_url}/docs\n")
        f.write(f"HEALTH_URL={clean_url}/\n")
        f.write(f"CHAT_URL={clean_url}/chat\n")

    print(f"""
    ✅ FIXED! Use this clean URL:

    🌐 API URL: {clean_url}
    📖 Documentation: {clean_url}/docs

    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL={clean_url}
    COLAB_ENABLED=true
    """)
else:
    print("❌ No active tunnels found")

🌐 Clean API URL: https://f56b-34-125-17-230.ngrok-free.app
INFO:     34.125.17.230:0 - "GET / HTTP/1.1" 200 OK
✅ API test successful!
   Response: {'status': 'healthy', 'model': 'phi3:mini', 'server': 'Google Colab', 'timestamp': '2025-06-13T22:03:29.431286'}

    ✅ FIXED! Use this clean URL:
    
    🌐 API URL: https://f56b-34-125-17-230.ngrok-free.app
    📖 Documentation: https://f56b-34-125-17-230.ngrok-free.app/docs
    
    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL=https://f56b-34-125-17-230.ngrok-free.app
    COLAB_ENABLED=true
    


In [ ]:
# Fix the ngrok URL format and test the API
from pyngrok import ngrok
import requests

# Get the clean URL
tunnels = ngrok.get_tunnels()
if tunnels:
    clean_url = tunnels[0].public_url
    print(f"🌐 Clean API URL: {clean_url}")

    # Test the API properly
    try:
        response = requests.get(f"{clean_url}/", timeout=10)
        if response.status_code == 200:
            print("✅ API test successful!")
            print(f"   Response: {response.json()}")
        else:
            print(f"⚠️ API returned status: {response.status_code}")
    except Exception as e:
        print(f"❌ API test failed: {e}")

    # Save the clean URL
    with open('api_url_clean.txt', 'w') as f:
        f.write(f"COLAB_API_URL={clean_url}\n")
        f.write(f"DOCS_URL={clean_url}/docs\n")
        f.write(f"HEALTH_URL={clean_url}/\n")
        f.write(f"CHAT_URL={clean_url}/chat\n")

    print(f"""
    ✅ FIXED! Use this clean URL:

    🌐 API URL: {clean_url}
    📖 Documentation: {clean_url}/docs

    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL={clean_url}
    COLAB_ENABLED=true
    """)
else:
    print("❌ No active tunnels found")

🌐 Clean API URL: https://f56b-34-125-17-230.ngrok-free.app
INFO:     34.125.17.230:0 - "GET / HTTP/1.1" 200 OK
✅ API test successful!
   Response: {'status': 'healthy', 'model': 'phi3:mini', 'server': 'Google Colab', 'timestamp': '2025-06-13T22:03:29.431286'}

    ✅ FIXED! Use this clean URL:
    
    🌐 API URL: https://f56b-34-125-17-230.ngrok-free.app
    📖 Documentation: https://f56b-34-125-17-230.ngrok-free.app/docs
    
    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL=https://f56b-34-125-17-230.ngrok-free.app
    COLAB_ENABLED=true
    


In [ ]:
# Fix the ngrok URL format and test the API
from pyngrok import ngrok
import requests

# Get the clean URL
tunnels = ngrok.get_tunnels()
if tunnels:
    clean_url = tunnels[0].public_url
    print(f"🌐 Clean API URL: {clean_url}")

    # Test the API properly
    try:
        response = requests.get(f"{clean_url}/", timeout=10)
        if response.status_code == 200:
            print("✅ API test successful!")
            print(f"   Response: {response.json()}")
        else:
            print(f"⚠️ API returned status: {response.status_code}")
    except Exception as e:
        print(f"❌ API test failed: {e}")

    # Save the clean URL
    with open('api_url_clean.txt', 'w') as f:
        f.write(f"COLAB_API_URL={clean_url}\n")
        f.write(f"DOCS_URL={clean_url}/docs\n")
        f.write(f"HEALTH_URL={clean_url}/\n")
        f.write(f"CHAT_URL={clean_url}/chat\n")

    print(f"""
    ✅ FIXED! Use this clean URL:

    🌐 API URL: {clean_url}
    📖 Documentation: {clean_url}/docs

    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL={clean_url}
    COLAB_ENABLED=true
    """)
else:
    print("❌ No active tunnels found")

🌐 Clean API URL: https://f56b-34-125-17-230.ngrok-free.app
INFO:     34.125.17.230:0 - "GET / HTTP/1.1" 200 OK
✅ API test successful!
   Response: {'status': 'healthy', 'model': 'phi3:mini', 'server': 'Google Colab', 'timestamp': '2025-06-13T22:03:29.431286'}

    ✅ FIXED! Use this clean URL:
    
    🌐 API URL: https://f56b-34-125-17-230.ngrok-free.app
    📖 Documentation: https://f56b-34-125-17-230.ngrok-free.app/docs
    
    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL=https://f56b-34-125-17-230.ngrok-free.app
    COLAB_ENABLED=true
    


Medical Chatbot Implementation

In [5]:
# ============================================================================
# GOOGLE COLAB MEDICAL CHATBOT - COMPLETE INTEGRATION
# Copy this entire code to your Google Colab notebook
# ============================================================================

import ollama
import json
import sqlite3
import uuid
from datetime import datetime
import re
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import Optional, List
import uvicorn
import threading
from pyngrok import ngrok
import time

# ============================================================================
# MEDICAL CHATBOT CLASS (Keep your existing class - it's perfect!)
# ============================================================================

class MedicalChatbotColab:
    def __init__(self):
        self.client = ollama.Client(host='http://127.0.0.1:11434')
        self.setup_database()
        self.medical_specialists = {
            'neurologue': ['tête', 'vertige', 'migraine', 'neurologique', 'mal de tête', 'cerveau', 'épilepsie', 'convulsion', 'paralysie', 'sclérose'],
            'cardiologue': ['cœur', 'poitrine', 'palpitation', 'cardiaque', 'thorax', 'hypertension', 'tension', 'infarctus', 'angine'],
            'gastro-entérologue': ['ventre', 'estomac', 'digestif', 'nausée', 'abdomen', 'diarrhée', 'constipation', 'vomissement', 'reflux', 'ulcère'],
            'dermatologue': ['peau', 'éruption', 'dermatologique', 'acné', 'eczéma', 'psoriasis', 'grain de beauté', 'démangeaison'],
            'gynécologue': ['menstruel', 'gynécologique', 'femme', 'règles', 'utérus', 'ovaire', 'sein', 'contraception', 'grossesse'],
            'urologue': ['urinaire', 'rein', 'vessie', 'prostate', 'infection urinaire', 'calcul', 'incontinence'],
            'pneumologue': ['respiration', 'poumon', 'toux', 'asthme', 'bronchite', 'essoufflement', 'pneumonie', 'allergie respiratoire'],
            'rhumatologue': ['articulation', 'arthrite', 'os', 'rhumatisme', 'arthrose', 'tendinite', 'fibromyalgie', 'lombalgie'],
            'endocrinologue': ['diabète', 'thyroïde', 'hormonal', 'sucre', 'obésité', 'cholestérol', 'métabolisme'],
            'psychiatre': ['anxiété', 'dépression', 'mental', 'stress', 'insomnie', 'panique', 'bipolarité', 'schizophrénie'],
            'orl': ['oreille', 'nez', 'gorge', 'sinusite', 'otite', 'surdité', 'acouphène', 'rhinite', 'angine'],
            'ophtalmologue': ['yeux', 'vision', 'vue', 'œil', 'cataracte', 'glaucome', 'myopie', 'conjonctivite'],
            'orthopédiste': ['fracture', 'entorse', 'luxation', 'genou', 'épaule', 'cheville', 'colonne vertébrale', 'prothèse'],
            'médecin généraliste': ['général', 'consultation', 'fatigue', 'fièvre', 'grippe', 'rhume', 'bilan', 'vaccin', 'check-up']
        }

    def setup_database(self):
        """Initialize SQLite database for Colab"""
        self.conn = sqlite3.connect('medical_chatbot.db', check_same_thread=False)
        cursor = self.conn.cursor()

        # Create chat_history table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS chat_history (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                conversation_id TEXT NOT NULL,
                patient_id TEXT NOT NULL,
                message TEXT NOT NULL,
                sender TEXT CHECK(sender IN ('user', 'assistant')) NOT NULL,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')

        # Create index for performance
        cursor.execute('''
            CREATE INDEX IF NOT EXISTS idx_conversation_patient
            ON chat_history(conversation_id, patient_id)
        ''')

        self.conn.commit()
        print("✅ Database initialized")

    def get_conversation_history(self, conversation_id, patient_id, limit=20):
        """Retrieve conversation history"""
        cursor = self.conn.cursor()
        cursor.execute('''
            SELECT message, sender, timestamp FROM chat_history
            WHERE conversation_id = ? AND patient_id = ?
            ORDER BY timestamp ASC LIMIT ?
        ''', (conversation_id, patient_id, limit))

        return cursor.fetchall()

    def save_message(self, conversation_id, patient_id, message, sender):
        """Save message to database"""
        cursor = self.conn.cursor()
        cursor.execute('''
            INSERT INTO chat_history (conversation_id, patient_id, message, sender)
            VALUES (?, ?, ?, ?)
        ''', (conversation_id, patient_id, message, sender))
        self.conn.commit()

    def detect_medical_specialty(self, message):
        """Detect appropriate medical specialist"""
        message_lower = message.lower()

        for specialist, keywords in self.medical_specialists.items():
            for keyword in keywords:
                if keyword in message_lower:
                    return specialist, f"pour les problèmes de {keyword}"

        return "médecin généraliste", "pour une consultation générale"

    def format_bold_text(self, text):
        """Convert markdown bold to HTML bold and add formatting to important medical terms"""
        # First convert existing markdown bold (**text**) to HTML bold (<strong>text</strong>)
        text = re.sub(r'\*\*(.*?)\*\*', r'<strong>\1</strong>', text)
        
        # Then ensure important medical terms are bolded if not already
        medical_terms = [
            'urgent', 'important', 'consultation', 'médecin', 'docteur',
            'symptômes', 'douleur', 'traitement', 'médicament', 'urgence',
            'neurologue', 'cardiologue', 'gastro-entérologue', 'dermatologue',
            'gynécologue', 'urologue', 'pneumologue', 'rhumatologue',
            'endocrinologue', 'psychiatre', 'orl', 'ophtalmologue',
            '24h', '48h', '72h', 'heures', 'jours', 'semaines', 'rappel', 'recommandation'
        ]

        for term in medical_terms:
            # Only bold if not already in strong tags
            pattern = re.compile(f'(?<!<strong>)\\b{re.escape(term)}\\b(?![^<]*</strong>)', re.IGNORECASE)
            text = pattern.sub(f'<strong>{term}</strong>', text)

        return text

    def is_first_message(self, conversation_id, patient_id):
        """Check if this is the first message in the conversation"""
        history = self.get_conversation_history(conversation_id, patient_id)
        return len(history) == 0

    def generate_medical_response(self, message, conversation_id, patient_id, language="fr"):
        """Generate medical response with context and recommendations"""
        try:
            # Get conversation history
            history = self.get_conversation_history(conversation_id, patient_id)
            is_first = self.is_first_message(conversation_id, patient_id)

            # Build conversation context - French only with enhanced prompts
            if is_first:
                system_prompt = """Tu es un assistant médical IA spécialisé en français. Pour le PREMIER message, tu dois:

1. Accueillir chaleureusement le patient
2. Expliquer ton rôle d'assistant médical
3. DEMANDER des informations spécifiques pour mieux aider:
   - Âge approximatif
   - Symptômes principaux et leur durée
   - Intensité de la douleur (si applicable) sur 10
   - Facteurs déclencheurs ou aggravants
   - Traitements déjà essayés
4. Recommander le type de médecin approprié
5. Inclure les disclaimers médicaux

Format pour le premier message:
- Accueil empathique
- Explication de ton rôle
- Questions spécifiques pour collecter les informations
- 👨‍⚕️ **Recommandation médicale**: [Spécialiste recommandé]
- ⚠️ **Rappel**: Consultez un **professionnel de santé**

IMPORTANT: Ne donne JAMAIS de diagnostic direct, seulement des conseils généraux."""
            else:
                system_prompt = """Tu es un assistant médical IA spécialisé en français. Pour les messages suivants, tu dois:

1. TOUJOURS inclure des disclaimers médicaux appropriés
2. Recommander des spécialistes médicaux quand nécessaire
3. Utiliser un formatage en gras pour les informations importantes
4. Ne JAMAIS donner de diagnostic direct
5. Être empathique et professionnel
6. Répondre UNIQUEMENT en français
7. Te souvenir de l'historique de conversation
8. Donner des conseils pratiques et utiles

Format de réponse souhaité:
- Réponse empathique à la question
- Conseils généraux appropriés et pratiques
- 👨‍⚕️ **Recommandation médicale**: [Spécialiste recommandé avec raison]
- ⚠️ **Rappel**: Consultez toujours un **professionnel de santé**"""

            context_messages = [{"role": "system", "content": system_prompt}]

            # Add conversation history
            for msg, sender, timestamp in history:
                role = "user" if sender == "user" else "assistant"
                context_messages.append({"role": role, "content": msg})

            # Add current message
            context_messages.append({"role": "user", "content": message})

            # Generate response using phi3:mini
            response = self.client.chat(
                model='phi3:mini',
                messages=context_messages,
                options={
                    "temperature": 0.7,
                    "top_p": 0.9,
                    "max_tokens": 500
                }
            )

            ai_response = response['message']['content']

            # Detect and add specialist recommendation if not present
            if "👨‍⚕️" not in ai_response:
                specialist, reason = self.detect_medical_specialty(message)
                # Always respond in French regardless of input language
                specialist_recommendation = f"\n\n👨‍⚕️ <strong>Recommandation médicale</strong>: Je vous conseille de consulter un <strong>{specialist}</strong> {reason}."
                ai_response += specialist_recommendation

            # Add medical disclaimer if not present (always in French)
            if "professionnel de santé" not in ai_response.lower():
                ai_response += "\n\n⚠️ <strong>Rappel</strong>: Cette conversation est à titre informatif uniquement. <strong>Consultez un professionnel de santé</strong> pour tout problème médical."

            # Format bold text
            ai_response = self.format_bold_text(ai_response)

            # Save messages to database
            self.save_message(conversation_id, patient_id, message, 'user')
            self.save_message(conversation_id, patient_id, ai_response, 'assistant')

            return {
                "response": ai_response,
                "conversation_id": conversation_id,
                "status": "success",
                "timestamp": datetime.now().isoformat()
            }

        except Exception as e:
            error_msg = f"Désolé, une erreur s'est produite: {str(e)}. Veuillez réessayer."
            return {
                "response": error_msg,
                "conversation_id": conversation_id,
                "status": "error",
                "timestamp": datetime.now().isoformat()
            }

# ============================================================================
# FASTAPI MODELS (Required for your Windows backend integration)
# ============================================================================

class ChatRequest(BaseModel):
    message: str
    conversation_id: Optional[str] = None
    patient_id: Optional[str] = "default_patient"

class ChatResponse(BaseModel):
    response: str
    conversation_id: str
    patient_id: str
    status: str
    timestamp: str

class HealthResponse(BaseModel):
    status: str
    model: str
    server: str
    timestamp: str

# ============================================================================
# FASTAPI APPLICATION (This is what your Windows app will connect to)
# ============================================================================

# Initialize the medical chatbot
medical_bot = MedicalChatbotColab()
print("✅ Medical Chatbot initialized")

# Create FastAPI app
app = FastAPI(
    title="Medical Chatbot API - Google Colab",
    description="AI Medical Assistant API powered by phi3:mini on Google Colab",
    version="1.0.0"
)

# Add CORS middleware for Windows app integration
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Configure this for production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/", response_model=HealthResponse)
async def health_check():
    """Health check endpoint - Your Windows app will use this to test connection"""
    return HealthResponse(
        status="healthy",
        model="phi3:mini",
        server="Google Colab",
        timestamp=datetime.now().isoformat()
    )

@app.post("/chat", response_model=ChatResponse)
async def chat_endpoint(request: ChatRequest):
    """Main chat endpoint - Your Windows app will send messages here"""
    try:
        # Generate conversation ID if not provided
        conversation_id = request.conversation_id or str(uuid.uuid4())

        # Generate response
        result = medical_bot.generate_medical_response(
            request.message,
            conversation_id,
            request.patient_id
        )

        if result["status"] == "success":
            return ChatResponse(
                response=result["response"],
                conversation_id=conversation_id,
                patient_id=request.patient_id,
                status="success",
                timestamp=result["timestamp"]
            )
        else:
            raise HTTPException(status_code=500, detail=result["response"])

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/conversations/{conversation_id}")
async def get_conversation_history(conversation_id: str, patient_id: str = "default_patient"):
    """Get conversation history - Your Windows app can retrieve chat history"""
    try:
        history = medical_bot.get_conversation_history(conversation_id, patient_id)
        return {
            "conversation_id": conversation_id,
            "patient_id": patient_id,
            "history": [
                {
                    "message": msg,
                    "sender": sender,
                    "timestamp": timestamp
                }
                for msg, sender, timestamp in history
            ]
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/reset-conversation")
async def reset_conversation():
    """Create new conversation ID"""
    new_conversation_id = str(uuid.uuid4())
    return {
        "conversation_id": new_conversation_id,
        "status": "success",
        "message": "New conversation started",
        "timestamp": datetime.now().isoformat()
    }

# ============================================================================
# SERVER STARTUP (This creates your API URL)
# ============================================================================

def start_api_server():
    """Start the FastAPI server and create public URL"""

    print("🚀 Starting Medical Chatbot API Server...")

    # Start FastAPI server in background
    def run_fastapi_server():
        uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

    # Start server in background thread
    server_thread = threading.Thread(target=run_fastapi_server, daemon=True)
    server_thread.start()

    # Wait for server to start
    print("⏳ Waiting for server to start...")
    time.sleep(8)

    # Create public URL with ngrok
    try:
        print("🌐 Setting up ngrok authentication...")

        # IMPORTANT: Replace 'YOUR_NGROK_AUTHTOKEN_HERE' with your actual ngrok authtoken
        # Get your authtoken from: https://dashboard.ngrok.com/get-started/your-authtoken
        NGROK_AUTHTOKEN = "2yTHefwwJgvXH834l9PEBvLdDfn_4bLeVXGdDd63rmgmgcLAd"  # ⚠️ REPLACE THIS WITH YOUR ACTUAL TOKEN

        if NGROK_AUTHTOKEN == "YOUR_NGROK_AUTHTOKEN_HERE":
            print("❌ ERROR: You need to replace 'YOUR_NGROK_AUTHTOKEN_HERE' with your actual ngrok authtoken!")
            print("📋 Steps to fix this:")
            print("1. Go to: https://dashboard.ngrok.com/get-started/your-authtoken")
            print("2. Copy your authtoken")
            print("3. Replace 'YOUR_NGROK_AUTHTOKEN_HERE' in the code above with your token")
            print("4. Re-run this cell")
            return None

        # Set the ngrok authtoken
        ngrok.set_auth_token(NGROK_AUTHTOKEN)
        print("✅ ngrok authentication configured")

        print("🌐 Creating public URL with ngrok...")
        public_url = ngrok.connect(8000)

        print(f"""
        ✅ SUCCESS! Your Medical Chatbot API is now running!

        📍 Local URL: http://localhost:8000
        🌐 Public URL: {public_url}
        📖 API Documentation: {public_url}/docs
        🔍 Health Check: {public_url}/
        💬 Chat Endpoint: {public_url}/chat

        ⚠️ IMPORTANT: Copy this URL to your Windows .env file:
        COLAB_API_URL={public_url}

        🔧 Test your API:
        curl {public_url}/
        """)

        # Save URL to file for easy access
        with open('api_url.txt', 'w') as f:
            f.write(f"COLAB_API_URL={public_url}\n")
            f.write(f"DOCS_URL={public_url}/docs\n")
            f.write(f"HEALTH_URL={public_url}/\n")
            f.write(f"CHAT_URL={public_url}/chat\n")

        print("💾 API URLs saved to 'api_url.txt' file")

        # Test the API to make sure it's working
        print("🧪 Testing API connection...")
        try:
            import requests
            response = requests.get(f"{public_url}/", timeout=10)
            if response.status_code == 200:
                print("✅ API test successful!")
                print(f"   Response: {response.json()}")
            else:
                print(f"⚠️ API test returned status code: {response.status_code}")
        except Exception as test_error:
            print(f"⚠️ API test failed: {test_error}")
            print("   This might be normal if the tunnel is still starting up")

        return public_url

    except Exception as e:
        print(f"❌ Error creating public URL: {e}")
        print("💡 Troubleshooting steps:")
        print("   1. Make sure you replaced 'YOUR_NGROK_AUTHTOKEN_HERE' with your actual token")
        print("   2. Check your internet connection")
        print("   3. Verify your ngrok authtoken is correct")
        print("   4. You can still use the local URL: http://localhost:8000")
        return None

# ============================================================================
# ALTERNATIVE: Manual ngrok setup function
# ============================================================================

def setup_ngrok_manually():
    """Alternative function to set up ngrok manually if the automatic setup fails"""
    print("🔧 Manual ngrok setup:")
    print("1. Replace the authtoken in the code above")
    print("2. Or run this in a new cell:")
    print("""
from pyngrok import ngrok

# Set your authtoken (replace with your actual token)
ngrok.set_auth_token("your_actual_authtoken_here")

# Create tunnel
public_url = ngrok.connect(8000)
print(f"🌐 Your API URL: {public_url}")

# Save to file
with open('api_url.txt', 'w') as f:
    f.write(f"COLAB_API_URL={public_url}\\n")

print("✅ Done! Copy the URL to your .env file")
    """)

# ============================================================================
# START THE API SERVER
# ============================================================================

# Start the server automatically
api_url = start_api_server()

if api_url:
    print(f"""
    🎉 Setup Complete!

    Next steps:
    1. Copy this URL: {api_url}
    2. Add to your Windows .env file: COLAB_API_URL={api_url}
    3. Set COLAB_ENABLED=true in your .env file
    4. Restart your Windows backend server
    5. Test the integration!

    ⚠️ Keep this Colab notebook running for the API to work!
    """)
else:
    print("""
    ⚠️ Public URL creation failed, but local server is running.

    🔧 To fix this:
    1. Replace 'YOUR_NGROK_AUTHTOKEN_HERE' with your actual ngrok authtoken
    2. Re-run this cell

    Or use the manual setup function: setup_ngrok_manually()
    """)

    # Show manual setup instructions
    setup_ngrok_manually()

✅ Database initialized
✅ Medical Chatbot initialized
🚀 Starting Medical Chatbot API Server...


INFO:     Started server process [2611]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


⏳ Waiting for server to start...
🌐 Setting up ngrok authentication...
✅ ngrok authentication configured
🌐 Creating public URL with ngrok...

        ✅ SUCCESS! Your Medical Chatbot API is now running!
        
        📍 Local URL: http://localhost:8000
        🌐 Public URL: NgrokTunnel: "https://f56b-34-125-17-230.ngrok-free.app" -> "http://localhost:8000"
        📖 API Documentation: NgrokTunnel: "https://f56b-34-125-17-230.ngrok-free.app" -> "http://localhost:8000"/docs
        🔍 Health Check: NgrokTunnel: "https://f56b-34-125-17-230.ngrok-free.app" -> "http://localhost:8000"/
        💬 Chat Endpoint: NgrokTunnel: "https://f56b-34-125-17-230.ngrok-free.app" -> "http://localhost:8000"/chat
        
        ⚠️ IMPORTANT: Copy this URL to your Windows .env file:
        COLAB_API_URL=NgrokTunnel: "https://f56b-34-125-17-230.ngrok-free.app" -> "http://localhost:8000"
        
        🔧 Test your API:
        curl NgrokTunnel: "https://f56b-34-125-17-230.ngrok-free.app" -> "http://localho

In [ ]:
# Fix the ngrok URL format and test the API
from pyngrok import ngrok
import requests

# Get the clean URL
tunnels = ngrok.get_tunnels()
if tunnels:
    clean_url = tunnels[0].public_url
    print(f"🌐 Clean API URL: {clean_url}")

    # Test the API properly
    try:
        response = requests.get(f"{clean_url}/", timeout=10)
        if response.status_code == 200:
            print("✅ API test successful!")
            print(f"   Response: {response.json()}")
        else:
            print(f"⚠️ API returned status: {response.status_code}")
    except Exception as e:
        print(f"❌ API test failed: {e}")

    # Save the clean URL
    with open('api_url_clean.txt', 'w') as f:
        f.write(f"COLAB_API_URL={clean_url}\n")
        f.write(f"DOCS_URL={clean_url}/docs\n")
        f.write(f"HEALTH_URL={clean_url}/\n")
        f.write(f"CHAT_URL={clean_url}/chat\n")

    print(f"""
    ✅ FIXED! Use this clean URL:

    🌐 API URL: {clean_url}
    📖 Documentation: {clean_url}/docs

    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL={clean_url}
    COLAB_ENABLED=true
    """)
else:
    print("❌ No active tunnels found")

🌐 Clean API URL: https://f56b-34-125-17-230.ngrok-free.app
INFO:     34.125.17.230:0 - "GET / HTTP/1.1" 200 OK
✅ API test successful!
   Response: {'status': 'healthy', 'model': 'phi3:mini', 'server': 'Google Colab', 'timestamp': '2025-06-13T22:03:29.431286'}

    ✅ FIXED! Use this clean URL:
    
    🌐 API URL: https://f56b-34-125-17-230.ngrok-free.app
    📖 Documentation: https://f56b-34-125-17-230.ngrok-free.app/docs
    
    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL=https://f56b-34-125-17-230.ngrok-free.app
    COLAB_ENABLED=true
    


In [6]:
# Fix the ngrok URL format and test the API
from pyngrok import ngrok
import requests

# Get the clean URL
tunnels = ngrok.get_tunnels()
if tunnels:
    clean_url = tunnels[0].public_url
    print(f"🌐 Clean API URL: {clean_url}")

    # Test the API properly
    try:
        response = requests.get(f"{clean_url}/", timeout=10)
        if response.status_code == 200:
            print("✅ API test successful!")
            print(f"   Response: {response.json()}")
        else:
            print(f"⚠️ API returned status: {response.status_code}")
    except Exception as e:
        print(f"❌ API test failed: {e}")

    # Save the clean URL
    with open('api_url_clean.txt', 'w') as f:
        f.write(f"COLAB_API_URL={clean_url}\n")
        f.write(f"DOCS_URL={clean_url}/docs\n")
        f.write(f"HEALTH_URL={clean_url}/\n")
        f.write(f"CHAT_URL={clean_url}/chat\n")

    print(f"""
    ✅ FIXED! Use this clean URL:

    🌐 API URL: {clean_url}
    📖 Documentation: {clean_url}/docs

    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL={clean_url}
    COLAB_ENABLED=true
    """)
else:
    print("❌ No active tunnels found")

🌐 Clean API URL: https://f56b-34-125-17-230.ngrok-free.app
INFO:     34.125.17.230:0 - "GET / HTTP/1.1" 200 OK
✅ API test successful!
   Response: {'status': 'healthy', 'model': 'phi3:mini', 'server': 'Google Colab', 'timestamp': '2025-06-13T22:03:29.431286'}

    ✅ FIXED! Use this clean URL:
    
    🌐 API URL: https://f56b-34-125-17-230.ngrok-free.app
    📖 Documentation: https://f56b-34-125-17-230.ngrok-free.app/docs
    
    📋 ADD THIS TO YOUR .ENV FILE:
    COLAB_API_URL=https://f56b-34-125-17-230.ngrok-free.app
    COLAB_ENABLED=true
    


In [ ]:
# ============================================================================
# ENHANCED TIMEOUT PREVENTION & SESSION MANAGEMENT
# ============================================================================

# Start the session manager to prevent timeouts
print("🔄 Starting enhanced session management...")
session_manager.start_keep_alive()

# Extend session for longer operation (total 50 minutes)
session_manager.extend_session(additional_minutes=30)

# Create a monitoring function
def monitor_session():
    """Monitor and display session information"""
    info = session_manager.get_session_info()
    print(f"""
    📊 SESSION STATUS:
    ⏰ Running for: {info['session_duration_minutes']:.1f} minutes
    ⏳ Time remaining: {info['remaining_time_minutes']:.1f} minutes
    🔄 Keep-alive active: {info['keep_alive_active']}
    📅 Last activity: {info['last_activity']}
    """)

# Display initial session info
monitor_session()

# Create a function to extend session if needed
def extend_session_time(minutes=20):
    """Extend the session by additional minutes"""
    session_manager.extend_session(minutes)
    print(f"⏰ Session extended by {minutes} minutes")
    monitor_session()

print("""
✅ Enhanced session management active!

🔧 Available commands:
• monitor_session() - Check current session status
• extend_session_time(30) - Extend session by 30 minutes
• session_manager.stop_keep_alive() - Stop keep-alive (if needed)

⚠️ IMPORTANT: This will help prevent timeouts, but Google Colab has limits:
• Free tier: ~12 hours max session
• Pro tier: ~24 hours max session
• Sessions may still timeout due to inactivity or resource limits
""")

# Test the enhanced API with longer timeout
print("\n🧪 Testing API with enhanced timeout handling...")
try:
    import requests
    from pyngrok import ngrok
    
    # Get the current tunnel URL
    tunnels = ngrok.get_tunnels()
    if tunnels:
        api_url = tunnels[0].public_url
        
        # Test with a longer timeout
        response = requests.get(f"{api_url}/", timeout=60)  # 60 second timeout
        if response.status_code == 200:
            print("✅ Enhanced API test successful!")
            print(f"   Response time: {response.elapsed.total_seconds():.2f} seconds")
        else:
            print(f"⚠️ API returned status: {response.status_code}")
    else:
        print("❌ No active tunnels found")
        
except Exception as e:
    print(f"❌ Enhanced API test failed: {e}")

print("\n🎯 TIMEOUT PREVENTION TIPS:")
print("1. Keep this notebook tab active in your browser")
print("2. Avoid closing the browser completely")
print("3. Run monitor_session() periodically to check status")
print("4. Extend session time before it expires")
print("5. If timeout occurs, re-run the main setup cell")
